In [183]:
#Importações 
import numpy as np
from PIL import Image

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms.v2 import Compose, ToImage, Normalize, \
Resize, ToPILImage, CenterCrop, RandomResizedCrop, ToDtype
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3

# Updated for Torchvision 0.15
from torchvision.models.alexnet import AlexNet_Weights

try:
    from torchvision.models.utils import load_state_dict_from_url
except ImportError:
    from torch.hub import load_state_dict_from_url

from stepbystep.v3 import StepByStep

In [184]:

# Função para calcular a acurácia de cada linha e a acurácia total
def calculate_accuracy(tensor_data):
    # Obter os resultados obtidos e esperados
    obtained = tensor_data[:, 0].float()
    expected = tensor_data[:, 1].float()
    
    # Calcular a porcentagem de acerto para cada linha
    accuracy_per_line = (obtained / expected) * 100
    
    # Calcular a acurácia total (média das acurácias individuais)
    total_accuracy = accuracy_per_line.mean()
    
    return accuracy_per_line, total_accuracy



In [185]:
# congelando os parâmtros da rede;
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad = False

A preparação do dataset que vai ser usado

In [186]:
# Adequando as imagens para o modelo AlexNet
normalizer = Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])

composer = Compose([Resize(256),
                    CenterCrop(224),
                    ToImage(), 
                    ToDtype(torch.float32, scale=True),
                    normalizer])

train_data = ImageFolder(root='yoga_train', transform=composer)
val_data = ImageFolder(root='yoga_test', transform=composer)


train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

In [187]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Instanciando o modelo que usado e passando os parâmetros padrão

Iniciando uma rede que todo a classificador é descongelado

In [188]:
weights = AlexNet_Weights.DEFAULT
alexnet_desfreeze = alexnet(weights=AlexNet_Weights.DEFAULT).to(device)

In [189]:
freeze_model(alexnet_desfreeze)

In [190]:
alexnet_desfreeze.classifier[6] = nn.Linear(4096,5)

In [191]:
# descongelando todo o classificador
for param in alexnet_desfreeze.classifier.parameters():
    param.requires_grad = True

In [192]:
# Exibindo a  camada descongelada
for name, param in alexnet_desfreeze.named_parameters():
    if param.requires_grad == True:
        print(name)

classifier.1.weight
classifier.1.bias
classifier.4.weight
classifier.4.bias
classifier.6.weight
classifier.6.bias


In [193]:
alexnet_desfreeze

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [194]:
# a função perda para essa situação é a CrossEntroyloss
# Já que é um problema de classificação multiclasse( e quando o modelo produz logits)
# E o otimizador usado foi o ADAM
torch.manual_seed(17)
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_alexnet_desfreeze = optim.Adam(alexnet_desfreeze.parameters(), lr=3e-4)

In [195]:
# neste caso estamos adicionando um modelo pronto como a alexnet_desfreeze numa arquitetura do autor do livro 
# a stepbystep
sbs_alexnet_desfreeze = StepByStep(alexnet_desfreeze, multi_loss_fn, optimizer_alexnet_desfreeze)
sbs_alexnet_desfreeze.set_loaders(train_loader, val_loader)
sbs_alexnet_desfreeze.train(5)

Avaliando o desempenho do Modelo que toda a camada de classificação aplicada a técnica do Fine-tuning

In [202]:
tensor_data_desfreeze=StepByStep.loader_apply(val_loader, sbs_alexnet_desfreeze.correct)
tensor_data_desfreeze

tensor([[ 73,  84],
        [104, 116],
        [ 90,  90],
        [ 54,  96],
        [  6, 109]])

In [204]:
# valor em porcentagem
accuracy_per_line, total_accuracy = calculate_accuracy(tensor_data_desfreeze)
print(f' A acuracia por classe {accuracy_per_line} % e a total foi de {total_accuracy} %')

 A acuracia por classe tensor([ 86.9048,  89.6552, 100.0000,  56.2500,   5.5046]) % e a total foi de 67.66290283203125 %


#### final da rede com toda a classificação descongelada

Testando com apenas última camada descongelada da etapa de classificação

In [205]:
weights = AlexNet_Weights.DEFAULT
alexnet_model = alexnet(weights=AlexNet_Weights.DEFAULT).to(device)


Aarquitetura da rede

In [206]:
alexnet_model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [207]:
#congelando toda a rede 
freeze_model(alexnet_model)

Treinando um modelo com apenas o topo "descongelado",os seus parâmetros são calculados

In [208]:
# Colocando a saída adequada para a classificação
# Por consequência descongela  a última camada do classificador que  
# terá os gradientes recebendo novos pesos
alexnet_model.classifier[6] = nn.Linear(4096,5)

In [209]:
# Exibindo a única camada descongelada
for name, param in alexnet_model.named_parameters():
    if param.requires_grad == True:
        print(name)

classifier.6.weight
classifier.6.bias


In [210]:
# a função perda para essa situação é a CrossEntroyloss
# Já que é um problema de classificação multiclasse( e quando o modelo produz logits)
# E o otimizador usado foi o ADAM
torch.manual_seed(17)
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_alexnet_model = optim.Adam(alexnet_model.parameters(), lr=3e-4)

In [211]:
# neste caso estamos adicionando um modelo pronto como a alexnet_model numa arquitetura do autor do livro 
# a stepbystep
sbs_alexnet_model = StepByStep(alexnet_model, multi_loss_fn, optimizer_alexnet_model)
sbs_alexnet_model.set_loaders(train_loader, val_loader)
sbs_alexnet_model.train(5)

Avaliando o desempenho do Modelo que  aplica a técnica do Fine-tuning em apenas 
uma cada, a última do classificador

In [212]:
tensor_data_1_desfreeze=StepByStep.loader_apply(val_loader, sbs_alexnet_model.correct)
tensor_data_1_desfreeze

tensor([[ 60,  84],
        [116, 116],
        [ 90,  90],
        [ 86,  96],
        [ 67, 109]])

In [214]:
# valor em porcentagem
accuracy_per_line, total_accuracy = calculate_accuracy(tensor_data_1_desfreeze)
print(f' A acuracia por classe {accuracy_per_line} % e a total foi de {total_accuracy} %')

 A acuracia por classe tensor([ 71.4286, 100.0000, 100.0000,  89.5833,  61.4679]) % e a total foi de 84.49595642089844 %


Fim das comparações das fine-tuning

Começa a features extract

Função para extrair as características e armazenar em um tensor e ser usado depois

In [215]:
# função necessária para extrair as caracteristicas
def preprocessed_dataset(model, loader, device=None):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if device is None:
        device = next(model.parameters()).device
    
    features = None
    labels = None

    for i, (x, y) in enumerate(loader):
        model.eval()
        x = x.to(device)
        output = model(x)
        if i == 0:
            features = output.detach().cpu()
            labels = y.cpu()
        else:
            features = torch.cat([features, output.detach().cpu()])
            labels = torch.cat([labels, y.cpu()])

    dataset = TensorDataset(features, labels)
    return dataset

Instanciando um novo modelo AlexNet

In [216]:
weights = AlexNet_Weights.DEFAULT
alexnet_feature_extractor= alexnet(weights=AlexNet_Weights.DEFAULT).to(device)

Extraindo as features

In [219]:
alexnet_feature_extractor.classifier[6] = nn.Identity()
alexnet_feature_extractor

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [220]:
train_preproc = preprocessed_dataset(alexnet_feature_extractor, train_loader)
val_preproc = preprocessed_dataset(alexnet_feature_extractor, val_loader)
train_preproc_loader = DataLoader(train_preproc, batch_size=16, shuffle=True)
val_preproc_loader = DataLoader(val_preproc, batch_size=16)

Treinando um modelo apenas com as features pré-processadas

In [221]:
torch.manual_seed(17)
top_alexnet_feature_extractor = nn.Sequential(nn.Linear(4096, 5))
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_top_alexnet_feature_extractor = optim.Adam(top_alexnet_feature_extractor.parameters(), lr=3e-4)

In [222]:
sbs_top = StepByStep(top_alexnet_feature_extractor, multi_loss_fn, optimizer_top_alexnet_feature_extractor)
sbs_top.set_loaders(train_preproc_loader, val_preproc_loader)
sbs_top.train(5)

In [225]:
tensor_data_feature_extractor=StepByStep.loader_apply(val_preproc_loader, sbs_top.correct)
tensor_data_feature_extractor

tensor([[ 46,  84],
        [116, 116],
        [ 84,  90],
        [ 40,  96],
        [ 85, 109]])

In [226]:
# Calcular a porcentagem de acerto de cada linha
accuracy_per_line, total_accuracy = calculate_accuracy(tensor_data_feature_extractor)
print(f' A acuracia por classe {accuracy_per_line} % e a total foi de {total_accuracy} %')


 A acuracia por classe tensor([ 54.7619, 100.0000,  93.3333,  41.6667,  77.9817]) % e a total foi de 73.54871368408203 %
